In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, plotly.express as px,os, nibabel as nib, cv2 as cv, tensorflow as tf, sklearn, tensorflow_addons as tfa
import PCA_Utilities
def Preprocessing_Steps(Pat_Dict,WG_Dict,Tumor_Dict):
    print("Initiating Data Cleanse")
    Pat_Dict,WG_Dict = PCA_Utilities.Preprocessing.Data_Normalization.Normalization(Pat_Dict,WG_Dict).Data_cleanse()
    print("Initiating Dilation of The whole gland")
    WG_Dict  = PCA_Utilities.Preprocessing.Data_Normalization.Normalization(WG_Dict,_).dilation_alg(20)
    print("Initiating the Distinction of the WG area from the image")
    Pat_Dict = PCA_Utilities.Preprocessing.Data_Normalization.Normalization(Pat_Dict,WG_Dict).Keep_area_of_interest()
    print("Initiating the cropping around the prostate process")
    Pat_Dict,Tumor_Dict = PCA_Utilities.Data_Normalization.Normalization(Pat_Dict,Tumor_Dict).crop()
    print("Initiating the Spatial resizing of patients and annotations")
    Pat_Dict   = PCA_Utilities.Data_Normalization.Normalization(Pat_Dict,_).resize(128,128,False)
    Tumor_Dict = PCA_Utilities.Data_Normalization.Normalization(Tumor_Dict,_).resize(128,128,True)
    print("Initiating 8Bit Normalization")
    Pat_Dict = PCA_Utilities.Data_Normalization.Normalization(Pat_Dict,_).norm8bit()
    print("Initiating the Axial resizing of patients and annotations")
    Pat_Dict   = PCA_Utilities.Data_Normalization.Normalization(Pat_Dict,_).resize_3D(16,128,128)
    Tumor_Dict = PCA_Utilities.Data_Normalization.Normalization(Tumor_Dict,_).resize_3D(16,128,128)
    return Pat_Dict,Tumor_Dict, WG_Dict

In [ ]:
# patients Loading
base_path = "/media/nst/HDD1/DIMITRIS_DEEP_LEARNING/DATASETS_UPDATED/DATASETS_PROSTATE/PROSTATE158/train"
T2,ADC,DWI, WG, _, _, Tumor_adc,Tumor_t2 = PCA_Utilities.Data_loaders.P158_load.Prostate158(base_path)

# A survey regarding the number of Lesion appeared in those patients
non_crit_pats = [key for key in T2.keys() if Tumor_adc[key].shape[0]==1]
crit_pats = [key for key in T2.keys() if (Tumor_adc[key].shape[0]>1 and Tumor_t2[key].shape[0]>1)]
#crit_pats.append("066")
print("Patients without ISUP<4: ",len(non_crit_pats))
print("Patients with Isup>=4: ",len(crit_pats))
a = [item for item in list(T2.keys()) if item not in non_crit_pats]
a = [item for item in a if item not in crit_pats]

In [ ]:
# perform Preprocessing operations
T2,Tumor_T2,WG_Dilated = Preprocessing_Steps(T2,WG,Tumor_t2)

In [ ]:
# keeping only patients with Tumor
T2_tums = {}
Tum_labs = {}
for key in Tumor_T2.keys():
    if len(Tumor_T2[key].shape) != 2:
        T2_tums.update({key:T2[key]})
        Tum_labs.update({key:Tumor_T2[key]})
        
# Creating The problem
tr_ls = list(T2_tums.keys())
tr_ls = tr_ls[:40]
val_ls= tr_ls[40:] 
T2_tums_train = {key: T2_tums[key] for key in tr_ls}
Tum_labs_train= {key: Tum_labs[key] for key in tr_ls}

T2_tums_val = {key: T2_tums[key] for key in val_ls}
Tum_labs_val= {key: Tum_labs[key] for key in val_ls}

T2_tr = {}
T2_vl = {}
for key in T2_tums_train:
    arr_3d = tf.keras.layers.Rescaling(1./255)(T2_tums_train[key])
    T2_tr.update({key:tf.expand_dims(arr_3d,axis=-1)})
for key in T2_tums_val:
    arr_3d = tf.keras.layers.Rescaling(1./255)(T2_tums_val[key])
    T2_vl.update({key:tf.expand_dims(arr_3d,axis=-1)})

In [ ]:
# Parameters for training
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()
SAVE_PATH = "WEIGHTS"

BATCH_SIZE = 2
STEPS_PER_EPOCH = len(T2_tr)//BATCH_SIZE
INIT_LR    = 0.0001
CLR        = tfa.optimizers.CyclicalLearningRate(initial_learning_rate = INIT_LR, maximal_learning_rate = 100*INIT_LR, scale_fn =lambda x:1/(2.**(x-1)),step_size=2 * STEPS_PER_EPOCH)
OPTIMIZER  = tf.optimizers.Adam(CLR)
LOSS       = tfa.losses.sigmoid_focal_crossentropy
EPOCHS     = 50
FILTERS    = [32,64,128,256,512,1024]
DenseModel = PCA_Utilities.Segmentation_Models.Models_3D.DenseUNET3D.DenseUNet3D(num_classes = 1, filters = FILTERS,num_convs=2)
DenseModel.training = True

DenseModel.compile(
    loss=LOSS,
    optimizer=OPTIMIZER,
    metrics=["acc"])

history = DenseModel.fit(x=np.array(list(T2_tr.values())),y=np.array(list(Tum_labs_train.values())), #[data_train,data_train,data_train]
                    validation_data=(np.array(list(T2_vl.values())),np.array(list(Tum_labs_val.values()))), #[data_val,data_val,data_val]
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    shuffle=True,
                    verbose=1,
                    callbacks=[PCA_Utilities.Classification_Models_3D.CallBacks.EarlyStoppingMinLoss(patience = 5)])
                    #PCA_Utilities.Classification_Models_3D.CallBacks.SaveModelH5(mod_nam = "DenseUnet", s_path = SAVE_PATH)])